# Workshop 04

<b>Obiettivo</b>: Convertire scripts da <i>classic PLaSM</i> a <i>Pyplasm</i> nel file <i>"CAD-LAMPS-2008.pdf"</i>.

In [1]:
# Importo la libreria:
import sys
sys.path.insert(0, 'C:\Users\Camilla Bianca\Documents\GGPL\lar-cc\larlib')
from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 0.00545439759667 seconds


<hr>
- <b>Colonna</b>:

In [ ]:
# PLASM:
DEF columna (dm, h , h b a s e : : isReal) = b a s e TOP
t o r u s b o t TOP c y l n d r TOP t o r u s t o p TOP c a p i t a l
TOP b a s e t o p
WHERE
c y l n d r = (JOIN∼TRUNCONE:<dm/2 , 0 . 8*dm/2 , h>) : 2 4 ,
t o r u s b o t = (JOIN∼TORUS:<dm/1 2 ,d/2>):<8 ,24>,
t o r u s t o p = (JOIN∼TORUS: <0.8*(dm/1 2 ) , 0 . 8 *(dm/
2 ) >):<8 ,24>,
b a s e = (T:<1,2>:<7*dm/-1 2 , 7*dm/-12> ∼ CUBOID)
:<7*dm/6 , 7*dm/6 , h b a s e >,
b a s e t o p = (T:<1,2>:<7*dm/-1 2 , 7*dm/-12> ∼
CUBOID) :<7*dm/6 , 7*dm/6 , dm/6>,
c a p i t a l = (JOIN ∼ TRUNCONE:< 0 . 8*dm/2 , 1 . 2*dm/
2 , h/8>) : 4 + (R: <1 ,2 >: (PI/4 ) ∼ JOIN ∼
TRUNCONE:< 0 . 8*dm/2 , 1 . 2*dm/2 , h/8>) : 
END ;

In [4]:
# PYPLASM:
def columna(dm,h,hbase):
    cylndr=JOIN(TRUNCONE([dm/2,0.8*dm/2,h])(24))
    torus_bot=JOIN(TORUS([dm/12,dm/2])([8,24]))
    torus_top=JOIN(TORUS([0.8*(dm/12),0.8*(dm/2)])([8,24]))
    base=T([1,2])([7*dm/-12,7*dm/-12])(CUBOID([7*dm/6,7*dm/6,dm/6]))
    base_top=T([1,2])([7*dm/-12,7*dm/-12])(CUBOID([7*dm/6,7*dm/6,dm/6]))
    capital=SUM([JOIN(TRUNCONE([0.8*dm/2,1.2*dm/2,h/8])(4)),R([1,2])(PI/4)(JOIN(TRUNCONE([0.8*dm/2,1.2*dm/2,h/8])(4)))])
    return TOP([TOP([TOP([TOP([TOP([base,torus_bot]),cylndr]),torus_top]),capital]),base_top])

VIEW(columna(6,36,2))

Child 3
Child 3


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E7B2570> >

![ws4_colonna.png](files/Immagini/ws4_colonna.png)

<hr>
- <b>Arco</b>:

In [ ]:
# PLASM:
DEF ArchSurface ( rr ,w::isreal) = Bezier:S2:< Circle0 , Circle1>
WHERE
Circle0 = <K: rr * cos ∼ S1 , K: rr * sin ∼ S1>,
Circle1 = <K:( rr - w) * cos ∼ S1 , K:( rr - w) * sin ∼ S1>
END ;

DEF Arch ( length ,w, depth ::isreal) ( angle ::isreal) = (T:3:(-: ceiling ) ∼ MAP: SolidMap) :domain3D
WHERE 
radius = ( length/2) / cos:( angle/2) ,
ceiling = MIN:2:(MAP: ArchSurf2D :domain2D) ,
domain2D = (T:1:( angle/2) ∼ intervals:(PI - angle ) ) :16 * q:1 ,
domain3D = domain2D * q:1 ,
SolidMap = Bezier:S3:<Surf3D_0 ,Surf3D_1 >, 
ArchSurf2D = ArchSurface:<radius ,w>,
Surf3D_0 = K:0 AL ArchSurf2D ,
Surf3D_1 = K: depth AL ArchSurf2D
END;

In [5]:
# PYPLASM:
def ArchSurface(rr,w,h):
    x1=lambda k: rr*cos(k[0])
    y1=lambda k: rr*sin(k[0])
    x2=lambda k: (rr-w)*cos(k[0])
    y2=lambda k: (rr-w)*sin(k[0])
    z=lambda k: h
    
    Circle0=BEZIER(S1)([CONS([x1,y1,z])])
    Circle1=BEZIER(S1)([CONS([x2,y2,z])])
    return BEZIER(S2)([Circle0,Circle1])

def Arch(length,w,depth,angle):
    radius=(length/2)/cos(angle/2)
    domain2D=PROD([T(1)(angle/2)(INTERVALS(PI-angle)(16)),QUOTE([1])])
    domain3D=PROD([domain2D,QUOTE([1])])
    ArchSurf2D_0=ArchSurface(radius,w,0)
    ArchSurf2D_1=ArchSurface(radius,w,depth)
    SolidMap=BEZIER(S3)([ArchSurf2D_0,ArchSurf2D_1])
    #ceiling=MIN(2)(MAP(ArchSurf2D)(domain2D))
    return MAP(SolidMap)(domain3D)

VIEW(Arch(20,1,2,0.2*PI))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E894630> >

![ws4_arco.png](files/Immagini/ws4_arco.png)

In [ ]:
# PLASM:
(STRUCT∼AA: (Arch:<3, 0.2, 0.4>)): (AA∼c:*:(pi/6)):(1..6))

In [ ]:
# PYPLASM: 

<hr>
- <b>Cupola</b>:

In [ ]:
# PLASM:
DEF HalfSphere (r::IsRealPos) = <fx , fy , fz>
WHERE
fx = K:r * - ∼ SIN ∼ S2 * COS ∼ S1 , fy = K:r * COS ∼ S1 * COS ∼ S2 , fz = K:r * SIN ∼ S1
END ;

DEF Dome (n::isnat) ( length ,w, angle ::isreal) = (T:3:(-: ceiling ) ∼ MAP: SolidMap) :domain3D
WHERE
radius = length/(2 * cos: angle ) ,
ceiling = MIN:3:dome1 ,
SolidMap = Bezier:S3:<Surf3D 0 ,
Surf3D 1 >, Surf3D 0 = HalfSphere : radius ,
Surf3D 1 = HalfSphere :( radius - w) ,
domain2D = (T:1:( angle ) ∼ Intervals:(PI angle ) ) :12 * Intervals:(2*PI) :n,
domain3D = domain2D * q:1
END;

In [6]:
# PYPLASM:
def HalfSphere(r):
    fx=lambda k: r*-(sin(k[1])*cos(k[0]))
    fy=lambda k: r*cos(k[0])*cos(k[1])
    fz=lambda k: r*sin(k[0])
    return CONS([fx,fy,fz])
                     
def Dome(n,length,w,angle):
    radius=length/(2*cos(angle))
    Surf3D_0=HalfSphere(radius)
    Surf3D_1=HalfSphere(radius-w)
    SolidMap=BEZIER(S3)([Surf3D_0,Surf3D_1])
    domain2D=T(1)(angle)(PROD([INTERVALS(PI-angle)(12),INTERVALS(2*PI)(n)]))
    domain3D=PROD([domain2D,QUOTE([1])])
    return MAP(SolidMap)(domain3D)
                     
VIEW(Dome(40,1,50,0.2*PI))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000E894DB0> >

![ws4_cupola.png](files/Immagini/ws4_cupola.png)